In [2]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [3]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
import json

conf = SparkConf()

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("Tatiana Gavrikova")
         .getOrCreate())

In [4]:
spark

In [5]:
sc = spark.sparkContext
sc

<SparkContext master=yarn appName=pyspark-shell>

In [7]:
rdd = sc.textFile('/labs/laba01/ml-100k/u.data')
rdd.take(2)

['196\t242\t3\t881250949', '186\t302\t3\t891717742']

In [8]:
rdd2 = rdd.map(lambda x: x.split())
rdd2.take(2)

[['196', '242', '3', '881250949'], ['186', '302', '3', '891717742']]

In [9]:
rdd3 = rdd2.map(lambda x: (x[1], x[2])).cache()
rdd3.take(2)

[('242', '3'), ('302', '3')]

In [10]:
hist_film = rdd3.filter(lambda x: x[0] == '302').map(lambda x: (x[1], 1)).countByKey().items()
hist_film

dict_items([('3', 46), ('4', 119), ('5', 120), ('2', 10), ('1', 2)])

In [11]:
hist_all = rdd3.map(lambda x: (x[1], 1)).countByKey().items()
hist_all

dict_items([('3', 27145), ('1', 6110), ('2', 11370), ('4', 34174), ('5', 21201)])

In [13]:
jsonObj = {
    'hist_film': [amount for _, amount in sorted(hist_film)],
    'hist_all': [amount for _, amount in sorted(hist_all)]
}

In [14]:
with open("lab01.json", "w") as outfile:
    json.dump(jsonObj, outfile)

In [15]:
!cat lab01.json

{"hist_film": [2, 10, 46, 119, 120], "hist_all": [6110, 11370, 27145, 34174, 21201]}

In [16]:
spark.stop()